In [207]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [1227]:
class Neuron:
    def __init__(self, input_shape):
      self.input_shape = input_shape
      self.bias = 1      
      self.bias_weight = np.random.uniform(low=-1, high=1.0, size=1)
      self.weight_matrix = np.random.uniform(low=-1, high=1.0, size=self.input_shape)

    def sigmoid(self,x):
      return 1/(1+np.exp(-x))
    
    def threshold(self, x):
      if x >= 0:
        return 1
      else:
        return -1
    
    def dot_product(self, input_pattern):
      return np.dot(input_pattern, self.weight_matrix)
        
    def activate(self, x, activation):
      if activation == 'linear':
        return x
      if activation == 'sigmoid':
        return self.sigmoid(x)
      if activation == 'tanh':
        return np.tanh(x)
      if activation == 'threshold':
        return self.threshold(x)

class Layer():
  def __init__(self, input_shape): 
    self.neurons = []
    self.input_shape = input_shape
    for i in range(input_shape[0]):
      self.neurons.append(Neuron(input_shape=input_shape[1]))
        
class MultiLayerPerceptron: 
  
  def build_layers(self, input_shape):
    for i in range(0, len(input_shape)-1):
      self.layers.append(Layer(input_shape=(input_shape[i+1],input_shape[i])))
  
  '''
      input_shape: Array or tuple with the corresponding
      number of neurons per layer.
      
  '''
  def __init__(self, input_shape):
    self.layers = [] # to_do replace layers by weight matrix
    self.build_layers(input_shape)
    self.weight_matrix = [None] * len(input_shape)
    self.weight_matrix[0] = np.random.uniform(low=-1, high=1.0, size=(input_shape[0]))
    
    # nao bolir
    for i in range(0, len(input_shape)-1):
      self.weight_matrix[i+1] = np.random.uniform(low=-1, high=1.0, size=(input_shape[i+1],input_shape[i])) # one column added to the corresponding bias weight
    for i in range(len(self.weight_matrix)):
      print(self.weight_matrix[i].shape)
    self.input_shape = input_shape

  def derivative(self, x, activation):
    if activation == 'tanh':
      return (1 - (np.tanh(x)**2))
    
    
  def forward_pass(self, input_pattern):
    first_out = []
    inact_out = []
    for neuron in self.layers[0].neurons:
      dot = neuron.dot_product(input_pattern) # neuron.bias * neuron.bias_weight[0] +
      induced_local_field = neuron.activate(dot, 'tanh')
      first_out.append(induced_local_field)
      inact_out.append(dot)
    output_by_layer = [first_out]
    inactivated_outputs = [inact_out]
    for i in range(len(self.layers)-1):
      layer_out = []
      inact_out = []
      for neuron in self.layers[i+1].neurons:
        dot = neuron.dot_product(inactivated_outputs[i]) # neuron.bias * neuron.bias_weight[0] +
        inact_out.append(dot)
        dot = neuron.dot_product(output_by_layer[i]) # neuron.bias * neuron.bias_weight[0] +
        induced_local_field = neuron.activate(dot, 'tanh')
        layer_out.append(induced_local_field)
      output_by_layer.append(layer_out)    
      inactivated_outputs.append(inact_out)
    return output_by_layer, inactivated_outputs 
  
  def backpropagate(self, output_by_layer, inact_output_by_layer, input_pattern, desired_output):
    error = []
    lr = 0.01 # test 
    deltas = []
    for l in reversed(range(len(self.layers))):
      # If it is an output layer:
      if (l+1) == len(self.layers):
        print('Output layer')
        for j in range(len(output_by_layer[l])):
          neuron_output = output_by_layer[l][j]
          error = (desired_output[j] - neuron_output) # (error**2)/2 
          print('Error by neuron:', error, 'OUTPUT_LAYER')
          # Update bias weight:
          current_neuron = self.layers[l].neurons[j] # truly neuron correspondent to this output?
          local_field_neuron_j = inact_output_by_layer[l][j]
          delta = error * self.derivative(local_field_neuron_j, 'tanh')
          deltas.append(delta)
          current_neuron.bias_weight = current_neuron.bias_weight + (lr * delta)
          # Update weights to previous layer neurons:
          for i in range(len(current_neuron.weight_matrix)):
            y_sub_i = output_by_layer[l-1][i]
            nabla_sub_ji = lr * delta * y_sub_i # nabla
            current_neuron.weight_matrix[i] = current_neuron.weight_matrix[i] + nabla_sub_ji
          # delta_weight = eta * delta_j(n) * y_i(n)
          # where delta_j(n) equals to: e_j(n) * fi'_j(v_j(n)) which corresponds to the local gradient
          # where y_i(n) equals to fi_i(v_i(n)) which corresponds to the input signal of neuron j           
      else:
        print('Hidden layer')
        # TODO: aggregate delta * weight (for each output neuron of the next (output) layer)
        for j in range(len(output_by_layer[l])): 
          accumulated_delta_k = 0
          # Code below assumes that the (next) layer l+1 is the output layer therefore should only work for 2 layers networks at first.
          # Despite that, making it generalizable should be straightforward since implies 
          # only testing whether layer 'l+1' is an output node or not (TO-DO).
          for k in range(len(output_by_layer[l+1])):  # For each neuron k (if output layer) do:
            neuron_output = output_by_layer[l+1][k]
            error = (desired_output[k] - neuron_output)
            local_field_neuron_k = inact_output_by_layer[l+1][k]
            delta_sub_k = error * self.derivative(local_field_neuron_k, 'tanh') # Get the weighted sum of the local gradients by the
            w_sub_kj = self.layers[l+1].neurons[k].weight_matrix[j] # corresponding weight connections between neurons j and k.
            accumulated_delta_k += delta_sub_k * w_sub_kj  # Propagate them back as error for updating the weights on neuron j.
          # Weight Update Rule:
          local_field_neuron_j = inact_output_by_layer[l][j]
          fi_sub_j = self.derivative(local_field_neuron_j, 'tanh')
          delta_j  =  fi_sub_j * accumulated_delta_k
          # Update weights:
          neuron_j = self.layers[l].neurons[j]
          for i in range(len(neuron_j.weight_matrix)):
            neuron_j.weight_matrix[i] = neuron_j.weight_matrix[i] + lr * (delta_j * input_pattern[i]) 
          # Update bias:
          neuron_j.bias_weight += delta_j * lr
        
  def train(X, y, self, learning_rate, error):
    prev_mse = 999999999
    current_mse  = 0
    epochs = 0 
    for (X,y) in zip(X,y):
      outputs = self.forward_pass(X)
      
      
nn = MultiLayerPerceptron(input_shape=[4,5,3])

(4,)
(5, 4)
(3, 5)


In [377]:
for w in nn.layers[0].neurons[0].weight_matrix:
  print(w)

0.44089024149141887
0.025968679754366786
0.36714037496661556
-0.1339619769057796


In [2017]:
X = data[0]['Iris-setosa'][0]
y = [0,0,1]
print('Input Pattern:', X, '| | Label:', y)

outputs, inactivated_outputs = nn.forward_pass(X)
print('Resulting Outputs:')
for output in outputs:
  print(output)

#print('Inactivated Outputs:')
#for output in inactivated_outputs:
#  print(output)
print('Backpropagation Result:')
nn.backpropagate(outputs, inactivated_outputs, X, y)

Input Pattern: [ 0.2820512  -0.12820512 -0.6666667  -0.974359  ] | | Label: [0, 0, 1]
Resulting Outputs:
[0.7852367670990879, -0.5153720282196315, 0.08652753109353586, 0.45180664753392447, -0.2395268600584649]
[0.0010951402662205709, 0.004581361034432474, 0.8875656329446858]
Backpropagation Result:
Output layer
Error by neuron: -0.0010951402662205709 OUTPUT_LAYER
Error by neuron: -0.004581361034432474 OUTPUT_LAYER
Error by neuron: 0.11243436705531418 OUTPUT_LAYER
Hidden layer


In [362]:
X = data[0]['Iris-setosa'][0]
y = [0,0,1]
print('Input Pattern:', X, '| | Label:', y)

outputs, inactivated_outputs = nn.forward_pass(X)
print('Resulting Outputs:')
for output in outputs:
  print(output)

#print('Inactivated Outputs:')
#for output in inactivated_outputs:
#  print(output)
print('Backpropagation Result:')
nn.backpropagate(outputs, inactivated_outputs, y)

Input Pattern: [ 0.2820512  -0.12820512 -0.6666667  -0.974359  ] | | Label: [0, 0, 1]
Resulting Outputs:
[0.00679099940227588, -0.28274649204076857, -0.40729772373546497, 0.5656305702283385, 0.41756191216861455]
[-0.7064888533882302, -0.18336722543214204, 0.32402345206121086]
Backpropagation Result:
Output layer
Error by neuron: 0.7064888533882302 OUTPUT_LAYER
Error by neuron: 0.18336722543214204 OUTPUT_LAYER
Error by neuron: 0.6759765479387891 OUTPUT_LAYER
Hidden layer


In [353]:
len(outputs[1])

3

In [320]:
nn.weight_matrix[1][0] = nn.layers[0].neurons[0].weight_matrix
nn.weight_matrix[1][1] = nn.layers[0].neurons[1].weight_matrix
nn.weight_matrix[1][2] = nn.layers[0].neurons[2].weight_matrix
nn.weight_matrix[1][3] = nn.layers[0].neurons[3].weight_matrix
nn.weight_matrix[1][4] = nn.layers[0].neurons[4].weight_matrix

nn.weight_matrix[2][0] = nn.layers[1].neurons[0].weight_matrix
nn.weight_matrix[2][1] = nn.layers[1].neurons[1].weight_matrix
nn.weight_matrix[2][2] = nn.layers[1].neurons[2].weight_matrix

test_neuron = Neuron(input_shape=1)
# forward pass function:
print('Input Layer:', X)
result = np.dot(X, np.transpose(nn.weight_matrix[1])) 
for i in range(len(result)):
  result[i] = test_neuron.activate(result[i], 'tanh')
print(result)
for i in range(2,len(nn.weight_matrix)):
  result = np.dot(result, np.transpose(nn.weight_matrix[i]))
  for j in range(len(result)):
    result[j] = test_neuron.activate(result[j], 'tanh')
  print(result)
  
#outputs, inactivated_ones = nn.forward_pass(nn.weight_matrix[0])
#print()
#print(inactivated_ones[0])
#print(inactivated_ones[1])
#print()
#print(outputs[0])
#print(outputs[1])



Input Layer: [ 0.2820512  -0.12820512 -0.6666667  -0.974359  ]
[0.22903411 0.48616465 0.74657338 0.33078536 0.23179094]
[-0.3170529   0.24037535 -0.61234509]


In [323]:
nn.layers[1].neurons[0].weight_matrix

add = 0
add += 

array([-0.67928981,  0.40334196, -0.80242887,  0.55023899,  0.20787583])

In [244]:
outputs, inactivated_ones = nn.forward_pass(nn.weight_matrix[0])
#nn.backpropagate(outputs, [1,0,0])

In [241]:
outputs[1]

[-0.9488529285876995, 0.7161474163933556, 0.9391050398666555]

In [230]:
for i in reversed(range(len(nn.layers))):
  print(i)

1
0


In [165]:
print(len(nn.layers[1].neurons))
print(nn.layers[1].neurons[0].weight_matrix.shape)

2
(5,)


In [297]:
nn.weight_matrix[1][0] = nn.layers[0].neurons[0].weight_matrix
nn.weight_matrix[1][1] = nn.layers[0].neurons[1].weight_matrix
nn.weight_matrix[1][2] = nn.layers[0].neurons[2].weight_matrix
nn.weight_matrix[1][3] = nn.layers[0].neurons[3].weight_matrix
nn.weight_matrix[1][4] = nn.layers[0].neurons[4].weight_matrix

nn.weight_matrix[2][0] = nn.layers[1].neurons[0].weight_matrix
nn.weight_matrix[2][1] = nn.layers[1].neurons[1].weight_matrix
nn.weight_matrix[2][2] = nn.layers[1].neurons[2].weight_matrix

test_neuron = Neuron(input_shape=1)
# forward pass function:
print('Input Layer:', nn.weight_matrix[0])
result = np.dot(nn.weight_matrix[0], np.transpose(nn.weight_matrix[1])) 
for i in range(len(result)):
  result[i] = test_neuron.activate(result[i], 'tanh')
print(result)
for i in range(2,len(nn.weight_matrix)):
  result = np.dot(result, np.transpose(nn.weight_matrix[i]))
  for j in range(len(result)):
    result[j] = test_neuron.activate(result[j], 'tanh')
  print(result)
  
outputs, inactivated_ones = nn.forward_pass(nn.weight_matrix[0])
print()
print(inactivated_ones[0])
print(inactivated_ones[1])
print()
print(outputs[0])
print(outputs[1])



Input Layer: [ 0.26779003 -0.50358615  0.71649754  0.01961462]
[-0.19018648  0.50754469  0.36764628  0.61928295 -0.49748821]
[ 0.01628897 -0.38813352 -0.55546591]

[-0.19253064122889776, 0.5594169350311841, 0.3856987876472758, 0.7238411246338803, -0.5459626807409129]
[-0.003956903094544939, -0.47267116986101954, -0.708851870955559]

[-0.19018648081251277, 0.5075446945177944, 0.36764627953494916, 0.6192829471613975, -0.4974882126298704]
[0.016288970116370247, -0.3881335179639093, -0.5554659123513682]


In [225]:
for neuron in nn.layers[0].neurons:
  print('Bias:' , neuron.bias)
  print('Bias weight:', neuron.bias_weight)
  print('Weight Matrix:', neuron.weight_matrix)

print()  
for neuron in nn.layers[1].neurons:

  print('Bias:' , neuron.bias)

  print('Bias weight:', neuron.bias_weight)

  print('Weight Matrix:', neuron.weight_matrix)  

Bias: 1
Bias weight: [0.2156437]
Weight Matrix: [0.5432315  0.72287656 0.39845131 0.0565957 ]
Bias: 1
Bias weight: [-0.2189839]
Weight Matrix: [ 0.07349027 -0.65203745 -0.82727309  0.78663128]
Bias: 1
Bias weight: [-0.43861804]
Weight Matrix: [-0.30502887  0.14722073  0.29790061 -0.31612596]
Bias: 1
Bias weight: [-0.50748182]
Weight Matrix: [ 0.83976118  0.53790922 -0.15006508  0.07148819]
Bias: 1
Bias weight: [-0.88297892]
Weight Matrix: [ 0.39303369  0.01601854  0.62880469 -0.94715056]

Bias: 1
Bias weight: [-0.56159472]
Weight Matrix: [ 0.21169708  0.31904389  0.11594083  0.7189823  -0.95576425]
Bias: 1
Bias weight: [0.8916593]
Weight Matrix: [-0.03179963  0.99742531 -0.89876418  0.13781569 -0.86104123]


In [183]:
nn.layers[0].neurons[0].bias_weight

array([0.93687414])

In [13]:
nn.weight_matrix[0]

array([[ 0.00111759,  0.67598335, -0.87129364, -0.48858705],
       [-0.61768803,  0.70156015, -0.88117519, -0.3185993 ],
       [-0.53684066, -0.04649365, -0.19035894, -0.03218577],
       [ 0.12676204, -0.18736079,  0.63508442, -0.68610225],
       [-0.83814944,  0.90275327, -0.13131897, -0.01142184]])

In [300]:
def assembly_dataset():
  f = open("iris.data", "r")
  data_x = []
  data_y = []
  for line in f:
    if len(line) != 1:
      data = line.replace('\n', '')
      t = data.split(',')
      data_y.append(t.pop(4))
      data_x.append(np.array(t,dtype=np.float32))
  return data_x, data_y

def normalize_row(row, max_val, min_val):
  for i in range(len(row)):
    row[i] = (2 * ((row[i] - min_val)/(max_val - min_val))) - 1 

def find_max_min(features):
  max_val = features[0][0]
  min_val = features[0][0]
  for i in range(len(features)):
    max_test = features[i][np.argmax(features[i])] 
    if max_test > max_val:
      max_val = max_test
      
    min_test = features[i][np.argmin(features[i])] 
    if min_test < min_val:
      min_val = min_test
  return max_val, min_val
    
def feature_normalization(features):
  max_val, min_val = find_max_min(features)
  for feature in features:
    normalize_row(feature, max_val, min_val)
    


In [301]:
def preprocess_dataset():  
  features, labels = assembly_dataset()
  feature_normalization(features)  

  data_dictionary = {'Iris-virginica' : [], 'Iris-versicolor': [], 'Iris-setosa': []}

  one_hot_label_dictionary = {'Iris-virginica' : [1,0,0], 'Iris-versicolor': [0,1,0], 'Iris-setosa': [0,0,1]}

  for t in zip(features,labels):
    data_dictionary[t[1]].append(t[0])
  return (data_dictionary, one_hot_label_dictionary)
data = preprocess_dataset()
print(data[0])
print(data[1])

{'Iris-virginica': [array([ 0.5897436 , -0.17948717,  0.5128205 , -0.38461536], dtype=float32), array([ 0.46153855, -0.3333333 ,  0.2820512 , -0.53846157], dtype=float32), array([ 0.7948718 , -0.25641024,  0.48717952, -0.48717952], dtype=float32), array([ 0.5897436 , -0.28205127,  0.4102564 , -0.5641026 ], dtype=float32), array([ 0.64102566, -0.25641024,  0.46153855, -0.46153843], dtype=float32), array([ 0.92307687, -0.25641024,  0.6666666 , -0.48717952], dtype=float32), array([ 0.23076928, -0.38461536,  0.12820518, -0.5897436 ], dtype=float32), array([ 0.84615386, -0.28205127,  0.5897436 , -0.5641026 ], dtype=float32), array([ 0.6923076 , -0.38461536,  0.46153855, -0.5641026 ], dtype=float32), array([ 0.8205128 , -0.1025641 ,  0.53846145, -0.38461536], dtype=float32), array([ 0.64102566, -0.2051282 ,  0.2820512 , -0.51282054], dtype=float32), array([ 0.6153846 , -0.3333333 ,  0.33333337, -0.53846157], dtype=float32), array([ 0.7179488 , -0.25641024,  0.38461542, -0.48717952], dtype=fl

In [302]:
data[0]['Iris-setosa']

[array([ 0.2820512 , -0.12820512, -0.6666667 , -0.974359  ], dtype=float32),
 array([ 0.23076928, -0.25641024, -0.6666667 , -0.974359  ], dtype=float32),
 array([ 0.17948711, -0.2051282 , -0.6923077 , -0.974359  ], dtype=float32),
 array([ 0.15384614, -0.23076928, -0.64102566, -0.974359  ], dtype=float32),
 array([ 0.25641024, -0.1025641 , -0.6666667 , -0.974359  ], dtype=float32),
 array([ 0.35897434, -0.02564102, -0.5897436 , -0.9230769 ], dtype=float32),
 array([ 0.15384614, -0.15384614, -0.6666667 , -0.94871795], dtype=float32),
 array([ 0.25641024, -0.15384614, -0.64102566, -0.974359  ], dtype=float32),
 array([ 0.1025641 , -0.28205127, -0.6666667 , -0.974359  ], dtype=float32),
 array([ 0.23076928, -0.23076928, -0.64102566, -1.        ], dtype=float32),
 array([ 0.35897434, -0.07692307, -0.64102566, -0.974359  ], dtype=float32),
 array([ 0.2051282 , -0.15384614, -0.61538464, -0.974359  ], dtype=float32),
 array([ 0.2051282 , -0.25641024, -0.6666667 , -1.        ], dtype=float32),

In [119]:
 # stratified random sampling
labels = list(data[1].keys())
X = []
y = []
for i in range(len(labels)):
  for x_sample in data[0][labels[i]]:
    X.append(x_sample)
    y.append(i)
    
skf = StratifiedKFold(n_splits=10) # shuffle and configure seed.
skf.get_n_splits(X, y)
for i, (train_index, test_index) in enumerate(skf.split(X, y)):
  print(f"Fold {i}:")
  print(f"  Train: index={train_index}")
  print(f"  Test:  index={test_index}")

Fold 0:
  Train: index=[  5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22
  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40
  41  42  43  44  45  46  47  48  49  55  56  57  58  59  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99
 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122
 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140
 141 142 143 144 145 146 147 148 149]
  Test:  index=[  0   1   2   3   4  50  51  52  53  54 100 101 102 103 104]
Fold 1:
  Train: index=[  0   1   2   3   4  10  11  12  13  14  15  16  17  18  19  20  21  22
  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40
  41  42  43  44  45  46  47  48  49  50  51  52  53  54  60  61  62  63
  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81
  82  83  84  85  86  87  88  89  90

In [116]:
for i in range(len(test_index)):
  print(y[test_index[i]])

0
0
0
0
0
1
1
1
1
1
2
2
2
2
2


In [77]:
feature_normalization(features)
#print(features)

524


IndexError: list index out of range

ValueError: too many values to unpack (expected 2)

TypeError: feature_normalization() missing 1 required positional argument: 'features'

In [53]:
data_dictionary

{'Iris-virginica': [['6.3', '3.3', '6.0', '2.5'],
  ['5.8', '2.7', '5.1', '1.9'],
  ['7.1', '3.0', '5.9', '2.1'],
  ['6.3', '2.9', '5.6', '1.8'],
  ['6.5', '3.0', '5.8', '2.2'],
  ['7.6', '3.0', '6.6', '2.1'],
  ['4.9', '2.5', '4.5', '1.7'],
  ['7.3', '2.9', '6.3', '1.8'],
  ['6.7', '2.5', '5.8', '1.8'],
  ['7.2', '3.6', '6.1', '2.5'],
  ['6.5', '3.2', '5.1', '2.0'],
  ['6.4', '2.7', '5.3', '1.9'],
  ['6.8', '3.0', '5.5', '2.1'],
  ['5.7', '2.5', '5.0', '2.0'],
  ['5.8', '2.8', '5.1', '2.4'],
  ['6.4', '3.2', '5.3', '2.3'],
  ['6.5', '3.0', '5.5', '1.8'],
  ['7.7', '3.8', '6.7', '2.2'],
  ['7.7', '2.6', '6.9', '2.3'],
  ['6.0', '2.2', '5.0', '1.5'],
  ['6.9', '3.2', '5.7', '2.3'],
  ['5.6', '2.8', '4.9', '2.0'],
  ['7.7', '2.8', '6.7', '2.0'],
  ['6.3', '2.7', '4.9', '1.8'],
  ['6.7', '3.3', '5.7', '2.1'],
  ['7.2', '3.2', '6.0', '1.8'],
  ['6.2', '2.8', '4.8', '1.8'],
  ['6.1', '3.0', '4.9', '1.8'],
  ['6.4', '2.8', '5.6', '2.1'],
  ['7.2', '3.0', '5.8', '1.6'],
  ['7.4', '2.8', '6.1'